## Načítanie knižníc a trénovacieho datasetu

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as ss
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import seaborn as sns
import math

In [2]:
df = pd.read_parquet("ml_dataset_train.parquet")

In [3]:
df.head()

,account_id,n_ad_reward_claims,n_ad_reward_fails,n_ads_watched,n_battlepass_lvls_finished,masked_feature_0,masked_feature_1,masked_feature_2,masked_feature_3,masked_feature_4,...,battlepass_22.0,battlepass_23.0,battlepass_8008.0,first_login_day_time,first_login_country_is_mfreq,time_to_first_purchase,time_to_last_purchase,time_between_last_purchase_last_login,sum_payments_package_key_ltv,n_payments_package_key_ltv
0,100000042,13,<NA>,26,8,175,354.0,354,281.0,141,...,False,False,False,eve,1,NaN,NaN,NaN,NaN,<NA>
1,100000053,<NA>,<NA>,7,1,25,17.0,17,8.0,8,...,False,False,False,morning,1,NaN,NaN,NaN,NaN,<NA>
2,100000082,<NA>,<NA>,<NA>,<NA>,1,NaN,<NA>,NaN,<NA>,...,False,False,False,eve,1,NaN,NaN,NaN,NaN,<NA>
3,100000112,9,<NA>,36,5,72,197.0,197,194.0,83,...,False,False,False,eve,1,NaN,NaN,NaN,NaN,<NA>
4,100000122,<NA>,<NA>,1,3,67,108.0,108,96.0,38,...,False,False,False,eve,1,NaN,NaN,NaN,NaN,<NA>


In [4]:
df.loc[:,["account_id","n_ads_watched","first_login_day_time","sum_payments_package_key","sum_payments_package_key_ltv"]].sort_values(by="sum_payments_package_key_ltv",ascending=False).head()

,account_id,n_ads_watched,first_login_day_time,sum_payments_package_key,sum_payments_package_key_ltv
2494459,169616730,3,early_morning,2854.17,22535.53
2508909,170207391,7,night,968.61,14256.66
2368471,165357101,1,noon,1446.35,11661.82
921994,127826834,<NA>,early_morning,984.62,7641.16
2145973,150404211,105,early_morning,63.89,6348.67


## Úprava datasetu a vyhodenie skrytých údajov(jednalo sa o citlivé údaje klientov)

In [5]:
empty_array = []
for i in range(0,42):
    empty_array.append("masked_feature_" +str(i))
print(empty_array)

['masked_feature_0', 'masked_feature_1', 'masked_feature_2', 'masked_feature_3', 'masked_feature_4', 'masked_feature_5', 'masked_feature_6', 'masked_feature_7', 'masked_feature_8', 'masked_feature_9', 'masked_feature_10', 'masked_feature_11', 'masked_feature_12', 'masked_feature_13', 'masked_feature_14', 'masked_feature_15', 'masked_feature_16', 'masked_feature_17', 'masked_feature_18', 'masked_feature_19', 'masked_feature_20', 'masked_feature_21', 'masked_feature_22', 'masked_feature_23', 'masked_feature_24', 'masked_feature_25', 'masked_feature_26', 'masked_feature_27', 'masked_feature_28', 'masked_feature_29', 'masked_feature_30', 'masked_feature_31', 'masked_feature_32', 'masked_feature_33', 'masked_feature_34', 'masked_feature_35', 'masked_feature_36', 'masked_feature_37', 'masked_feature_38', 'masked_feature_39', 'masked_feature_40', 'masked_feature_41']


In [6]:
df.drop(['masked_feature_0', 'masked_feature_1', 'masked_feature_2', 'masked_feature_3', 'masked_feature_4', 'masked_feature_5', 'masked_feature_6', 'masked_feature_7', 'masked_feature_8', 'masked_feature_9','masked_feature_13','masked_feature_14','masked_feature_15', 'masked_feature_16', 'masked_feature_17', 'masked_feature_18', 'masked_feature_20', 'masked_feature_21', 'masked_feature_22', 'masked_feature_23', 'masked_feature_24', 'masked_feature_25', 'masked_feature_26', 'masked_feature_27', 'masked_feature_28', 'masked_feature_29', 'masked_feature_30', 'masked_feature_31', 'masked_feature_32', 'masked_feature_33', 'masked_feature_34', 'masked_feature_35', 'masked_feature_36', 'masked_feature_37', 'masked_feature_38', 'masked_feature_39', 'masked_feature_40', 'masked_feature_41'],axis=1,inplace=True)

## Rozdelenie a úprava datasetu pre ďalší postup a popisná štatistika jednotlivých častí pre lepší prehľad

In [7]:
# rozdelenie datasetu na buyerov a non_buyerov
buyers = df.loc[(df["sum_payments_package_key_ltv"]>0)]
non_buyers = df.loc[(df["n_payments_package_key_ltv"].isna())]
buyers_count = len(buyers.index)
non_buyers_count = len(non_buyers.index)
buyers_non_buyers_ratio = round((buyers_count/non_buyers_count)*100,2)
print(f"Percento buyerov v datasete {buyers_non_buyers_ratio}")

Percento buyerov v datasete 2.55


In [8]:
#oddelenie numerických premenných pre buyerov od tých ostatných nakoľko v regresných modeloch budeme pracovať len s numerickými premennými
numeric_columns_buyers = buyers.dtypes[buyers.dtypes!=np.object_].index.tolist()
numeric_variables_buyers = buyers.loc[:,numeric_columns_buyers].astype("float64")
# nahradenie nan hodnôt priemerom príslušných stĺpcov a zbavenie sa boolean hodnot+následne resetnutie indexu aby sme boli schopní pristupovať k dátam 
# cez index
numeric_variables_buyers.drop(["battlepass_0.0","battlepass_22.0","battlepass_23.0","battlepass_8008.0"],inplace=True,axis=1)

In [9]:
# nahradenie nan hodnôt priemerom príslušných stĺpcov a zbavenie sa boolean hodnot+následne resetnutie indexu aby sme boli schopní pristupovať k dátam 
# cez index
for i in range(numeric_variables_buyers.shape[1]):
    numeric_variables_buyers.iloc[:,i].fillna(numeric_variables_buyers.iloc[:,i].mean(),inplace=True)
numeric_variables_buyers.reset_index(inplace=True,drop=True)

In [10]:
#Vyfiltrovanie textových premenných pre buyerov
string_columns_buyers = buyers.dtypes[buyers.dtypes==np.object_].index.tolist()
string_variables_buyers= buyers.loc[:,string_columns_buyers]

In [11]:
#popisná štatistika textových premenných pre buyerov
string_variables_buyers.describe() 

,account_id,device_language,first_device_os,first_time_zone,first_device_model,first_network_type,first_login_country,first_device_manufacturer,is_paid_user,form_factor,most_frequent_country,most_frequent_network_type,first_login_weekday,first_login_day_time
count,69858,69841,69841,69841,69841,69841,69841,69841,69841,31040,69841,69841,69858,69858
unique,69858,43,3,32,2182,4,174,117,2,3,174,5,7,6
top,100000423,en,android,+09:00,"iPhone12,1",WIFI,US,Apple,True,Phone,US,WIFI,Sunday,noon
freq,1,36957,35701,9722,5448,45020,20297,34140,65428,30334,20304,43849,11098,14159


In [12]:
#vyfiltrovanie číselných premenných pre non-buyerov
numeric_columns_non_buyers = non_buyers.dtypes[non_buyers.dtypes!=np.object_].index.tolist()
numeric_variables_non_buyers = non_buyers.loc[:,numeric_columns_non_buyers]

In [13]:
#popisná štatistika číselných premenných pre non-buyerov
numeric_variables_non_buyers.describe()

,n_ad_reward_claims,n_ad_reward_fails,n_ads_watched,n_battlepass_lvls_finished,n_instant_awards_claims,n_extra_challenge_buys,connected_fb,connect_fb_attempt,n_friend_installs,n_accept_all_friends,...,screen_height,nunique_countries,nunique_network_types,nunique_iaps_bought,first_login_country_is_mfreq,time_to_first_purchase,time_to_last_purchase,time_between_last_purchase_last_login,sum_payments_package_key_ltv,n_payments_package_key_ltv
count,225719.000000,182300.000000,924496.000000,758512.000000,6923.000000,10646.000000,106847.000000,144687.000000,1463.000000,26906.000000,...,1.147467e+06,2.742969e+06,2.742969e+06,1282.000000,2.742985e+06,0.0,0.0,0.0,0.0,0
mean,6.681932,1.837926,20.405868,3.516323,1.472916,1.115536,1.106114,1.353515,1.101846,1.580354,...,2.004614e+03,1.010239e+00,1.350424e+00,1.054602,9.972658e-01,NaN,NaN,NaN,NaN,<NA>
std,9.125862,2.492890,34.280043,2.042770,0.864203,0.378111,0.418406,0.890160,0.403325,1.529549,...,3.724043e+02,1.142600e-01,5.387187e-01,0.355812,5.221851e-02,NaN,NaN,NaN,NaN,<NA>
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,5.000000e+00,1.000000e+00,1.000000e+00,1.000000,0.000000e+00,NaN,NaN,NaN,NaN,<NA>
25%,4.000000,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.640000e+03,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,NaN,NaN,NaN,NaN,<NA>
50%,5.000000,1.000000,7.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2.009000e+03,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,NaN,NaN,NaN,NaN,<NA>
75%,9.000000,2.000000,22.000000,5.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,...,2.340000e+03,1.000000e+00,2.000000e+00,1.000000,1.000000e+00,NaN,NaN,NaN,NaN,<NA>
max,3038.000000,201.000000,617.000000,59.000000,26.000000,6.000000,22.000000,38.000000,7.000000,71.000000,...,3.840000e+03,9.000000e+00,5.000000e+00,8.000000,1.000000e+00,NaN,NaN,NaN,NaN,<NA>


In [14]:
# Vyfiltrovanie textových premenných pre non-buyerov
string_columns_non_buyers = non_buyers.dtypes[non_buyers.dtypes==np.object_].index.tolist()
string_variables_non_buyers = non_buyers.loc[:,string_columns_non_buyers]

In [15]:
# Popisná štatistika textových premenných pre non-buyerov
string_variables_non_buyers.describe()

,account_id,device_language,first_device_os,first_time_zone,first_device_model,first_network_type,first_login_country,first_device_manufacturer,is_paid_user,form_factor,most_frequent_country,most_frequent_network_type,first_login_weekday,first_login_day_time
count,2742985,2742969,2742985,2742985,2742985,2742985,2742979,2742985,2742637,1147467,2742969,2742969,2742985,2742985
unique,2742985,122,3,39,10438,7,230,1180,2,3,231,6,7,6
top,100000042,en,android,+01:00,"iPhone12,1",WIFI,US,Apple,False,Phone,US,WIFI,Sunday,noon
freq,1,1377856,1440797,376185,233898,1928017,605961,1302189,2738109,1099691,605395,1897433,427703,586142


## Načítanie a úprava testovacieho datasetu

In [16]:
df_test = pd.read_parquet("ml_dataset_test.parquet")
df_test.drop(['masked_feature_0', 'masked_feature_1', 'masked_feature_2', 'masked_feature_3', 'masked_feature_4', 'masked_feature_5', 'masked_feature_6', 'masked_feature_7', 'masked_feature_8', 'masked_feature_9','masked_feature_13','masked_feature_14','masked_feature_15', 'masked_feature_16', 'masked_feature_17', 'masked_feature_18', 'masked_feature_20', 'masked_feature_21', 'masked_feature_22', 'masked_feature_23', 'masked_feature_24', 'masked_feature_25', 'masked_feature_26', 'masked_feature_27', 'masked_feature_28', 'masked_feature_29', 'masked_feature_30', 'masked_feature_31', 'masked_feature_32', 'masked_feature_33', 'masked_feature_34', 'masked_feature_35', 'masked_feature_36', 'masked_feature_37', 'masked_feature_38', 'masked_feature_39', 'masked_feature_40', 'masked_feature_41'],axis=1,inplace=True)
df_test.head()

,account_id,n_ad_reward_claims,n_ad_reward_fails,n_ads_watched,n_battlepass_lvls_finished,n_instant_awards_claims,n_extra_challenge_buys,connected_fb,connect_fb_attempt,n_friend_installs,...,battlepass_22.0,battlepass_23.0,battlepass_8008.0,first_login_day_time,first_login_country_is_mfreq,time_to_first_purchase,time_to_last_purchase,time_between_last_purchase_last_login,sum_payments_package_key_ltv,n_payments_package_key_ltv
12,100000293,<NA>,<NA>,<NA>,2,<NA>,<NA>,2,2,<NA>,...,False,False,False,morning,1,NaN,NaN,NaN,NaN,<NA>
13,100000333,<NA>,<NA>,5,2,<NA>,<NA>,<NA>,<NA>,<NA>,...,False,False,False,morning,1,NaN,NaN,NaN,NaN,<NA>
19,100000463,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,False,False,False,morning,1,NaN,NaN,NaN,NaN,<NA>
24,100000624,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,False,False,False,eve,1,NaN,NaN,NaN,NaN,<NA>
28,100000713,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,False,False,False,morning,1,NaN,NaN,NaN,NaN,<NA>


In [17]:
buyers_test = df_test.loc[(df_test["sum_payments_package_key_ltv"].notna())]
non_buyers_test = df_test.loc[(df_test["n_payments_package_key_ltv"].isna())]
buyers_count_test = len(buyers_test.index)
non_buyers_count_test = len(non_buyers_test.index)
buyers_non_buyers_ratio_test = round((buyers_count_test/non_buyers_count_test)*100,2)
buyers_non_buyers_ratio_test

2.55

In [48]:
#vyfiltrovanie numerickych premennych
buyers_test_numeric_columns = buyers_test.dtypes[buyers_test.dtypes!=np.object_].index.tolist()
buyers_test_numeric_variables = buyers_test.loc[:,buyers_test_numeric_columns].astype("float64")
# odstraneie boolean premenných pre zjednodušenie procesu a nahradenie nan honotami nulov + resetnutie indexu
buyers_test_numeric_variables.drop(["battlepass_0.0","battlepass_22.0","battlepass_23.0","battlepass_8008.0"],axis=1,inplace = True)
#buyers_test_numeric_variables.fillna(0,inplace = True)
#buyers_test_numeric_variables.reset_index(inplace=True,drop=True)
#buyers_test_numeric_variables.head() 

In [56]:
# v testovacom boli vsetky hodnoty NaN, preto so vyhodil v oboch datasetoch
buyers_test_numeric_variables.drop(["max_session_end_viplevel"],axis=1,inplace=True)
numeric_variables_buyers.drop(["max_session_end_viplevel"],axis=1,inplace=True)

KeyError: "['max_session_end_viplevel'] not found in axis"

In [57]:
for i in range(buyers_test_numeric_variables.shape[1]):
    buyers_test_numeric_variables.iloc[:,i].fillna(buyers_test_numeric_variables.iloc[:,i].mean(),inplace=True)
buyers_test_numeric_variables.reset_index(inplace=True,drop=True)

In [21]:
buyers_test_numeric_variables.describe()

,n_ad_reward_claims,n_ad_reward_fails,n_ads_watched,n_battlepass_lvls_finished,n_instant_awards_claims,n_extra_challenge_buys,connected_fb,connect_fb_attempt,n_friend_installs,n_accept_all_friends,...,screen_height,nunique_countries,nunique_network_types,nunique_iaps_bought,first_login_country_is_mfreq,time_to_first_purchase,time_to_last_purchase,time_between_last_purchase_last_login,sum_payments_package_key_ltv,n_payments_package_key_ltv
count,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000,...,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000
mean,9.040816,2.302998,39.246882,5.493306,1.795134,1.357815,1.130055,1.385053,1.103448,1.879132,...,2111.355398,1.018711,1.779038,2.162364,0.995869,47933.866898,79916.267889,128851.672309,39.847188,4.815097
std,4.235898,1.452575,46.083714,2.551292,0.252834,0.192188,0.197628,0.385524,0.027563,0.473573,...,190.181631,0.148615,0.605821,2.425169,0.064139,62807.728145,79613.747689,83688.356096,198.594481,10.309769
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,854.000000,1.000000,1.000000,1.000000,0.000000,13.048000,16.568000,0.411000,0.990000,1.000000
25%,8.000000,2.302998,8.000000,4.000000,1.795134,1.357815,1.130055,1.385053,1.103448,1.879132,...,2009.000000,1.000000,1.000000,1.000000,1.000000,3762.480500,7341.492500,54920.091500,1.995000,1.000000
50%,9.040816,2.302998,26.000000,5.493306,1.795134,1.357815,1.130055,1.385053,1.103448,1.879132,...,2111.355398,1.000000,2.000000,1.000000,1.000000,14875.246000,52190.360000,134973.114000,4.990000,2.000000
75%,9.040816,2.302998,47.000000,7.000000,1.795134,1.357815,1.130055,1.385053,1.103448,1.879132,...,2111.355398,1.000000,2.000000,2.000000,1.000000,75674.874500,140535.069500,204529.020000,22.970000,4.000000
max,44.000000,39.000000,635.000000,50.000000,10.000000,8.000000,11.000000,13.000000,3.000000,16.000000,...,3200.000000,4.000000,5.000000,47.000000,1.000000,259119.486000,259197.446000,258530.907000,11701.320000,368.000000


## Lineárna regresia za pomoci statsmodels

In [22]:
X_train = numeric_variables_buyers.iloc[:,:45]
Y_train = numeric_variables_buyers.iloc[:,45].to_frame()

In [23]:
X_test = buyers_test_numeric_variables.iloc[:,:45]
Y_test = buyers_test_numeric_variables.iloc[:,45].to_frame()

In [24]:
dep_var = Y_train
exp_var = X_train
exp_var = sm.add_constant(exp_var) 

In [25]:
ols = sm.OLS(dep_var.astype(float),exp_var.astype(float))
ols_fit = ols.fit()
ols_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     sum_payments_package_key_ltv   R-squared:                       0.429
Model:                                      OLS   Adj. R-squared:                  0.428
Method:                           Least Squares   F-statistic:                     1164.
Date:                          Wed, 18 May 2022   Prob (F-statistic):               0.00
Time:                                  16:56:44   Log-Likelihood:            -4.4724e+05
No. Observations:                         69858   AIC:                         8.946e+05
Df Residuals:                             69812   BIC:                         8.950e+05
Df Model:                                    45                                         
Covariance Type:                      nonrobust                                         
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
const                                    36.9043     60.356      0.611      0.541     -81.392     155.201
n_ad_reward_claims                        0.1750      0.149      1.171      0.242      -0.118       0.468
n_ad_reward_fails                         0.3286      0.391      0.841      0.401      -0.438       1.095
n_ads_watched                            -0.0224      0.015     -1.478      0.139      -0.052       0.007
n_battlepass_lvls_finished               -0.9839      0.366     -2.687      0.007      -1.702      -0.266
n_instant_awards_claims                  -5.4216      2.425     -2.236      0.025     -10.174      -0.669
n_extra_challenge_buys                   10.5771      2.691      3.930      0.000       5.302      15.852
connected_fb                              0.0673      3.185      0.021      0.983      -6.175       6.309
connect_fb_attempt                       -2.5284      1.738     -1.455      0.146      -5.934       0.877
n_friend_installs                        10.3892     14.753      0.704      0.481     -18.526      39.305
n_accept_all_friends                      0.4300      1.153      0.373      0.709      -1.830       2.690
n_accept_friend                          -8.7799      1.719     -5.107      0.000     -12.149      -5.410
n_clicked_add_friends                    -0.9783      0.622     -1.573      0.116      -2.198       0.241
n_friend_searches                        -2.6886      3.416     -0.787      0.431      -9.383       4.006
n_levels_completed                       -0.4956      0.183     -2.707      0.007      -0.855      -0.137
n_calendar_login_days                    -0.1302      0.075     -1.727      0.084      -0.278       0.018
n_missions_completed                      0.0330      0.010      3.168      0.002       0.013       0.053
n_package_info_offers_viewed             -0.0052      0.001     -4.296      0.000      -0.008      -0.003
n_payments_package_key                    0.4248      0.721      0.589      0.556      -0.989       1.838
sum_payments_package_key                  2.6645      0.021    129.203      0.000       2.624       2.705
n_package_tips_offers_viewed             -0.1957      0.034     -5.771      0.000      -0.262      -0.129
rated_game                              -72.8182     51.704     -1.408      0.159    -174.158      28.522
n_remove_ad_clicks                       -0.5076      1.476     -0.344      0.731      -3.401       2.386
n_sessions_ended                         -0.5028      0.152     -3.319      0.001      -0.800      -0.206
total_session_duration                 6.611e-07   4.36e-08     15.166      0.000    5.76e-07    7.47e-07
max_s

In [26]:
adjusted_X_train = X_train.drop(["n_ad_reward_claims","n_ad_reward_fails","n_ads_watched","connected_fb","connect_fb_attempt","n_friend_installs","n_accept_all_friends","n_clicked_add_friends","n_friend_searches","n_calendar_login_days","n_payments_package_key","rated_game","n_remove_ad_clicks","max_session_end_player_level","min_session_start_viplevel","bigmac_dollar_price","screen_density","screen_width","screen_height","nunique_countries","nunique_network_types","first_login_country_is_mfreq","time_between_last_purchase_last_login"],axis=1)
adjusted_X_test = X_test.drop(["n_ad_reward_claims","n_ad_reward_fails","n_ads_watched","connected_fb","connect_fb_attempt","n_friend_installs","n_accept_all_friends","n_clicked_add_friends","n_friend_searches","n_calendar_login_days","n_payments_package_key","rated_game","n_remove_ad_clicks","max_session_end_player_level","min_session_start_viplevel","bigmac_dollar_price","screen_density","screen_width","screen_height","nunique_countries","nunique_network_types","first_login_country_is_mfreq","time_between_last_purchase_last_login"],axis=1)

In [27]:
adj_dep_var = Y_train
adj_exp_var = adjusted_X_train

In [28]:
adj_ols = sm.OLS(adj_dep_var.astype(float),adj_exp_var.astype(float))
adj_ols_fit = adj_ols.fit()
adj_ols_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      OLS Regression Results                                     
=================================================================================================
Dep. Variable:     sum_payments_package_key_ltv   R-squared (uncentered):                   0.452
Model:                                      OLS   Adj. R-squared (uncentered):              0.452
Method:                           Least Squares   F-statistic:                              2620.
Date:                          Wed, 18 May 2022   Prob (F-statistic):                        0.00
Time:                                  16:56:49   Log-Likelihood:                     -4.4726e+05
No. Observations:                         69858   AIC:                                  8.946e+05
Df Residuals:                             69836   BIC:                                  8.948e+05
Df Model:                                    22                                                  
Covariance Type:                      nonrobust                                                  
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
n_battlepass_lvls_finished        -1.0999      0.351     -3.130      0.002      -1.789      -0.411
n_instant_awards_claims           -8.5511      1.998     -4.281      0.000     -12.467      -4.636
n_extra_challenge_buys             7.5685      2.335      3.242      0.001       2.993      12.144
n_accept_friend                  -10.0592      1.583     -6.356      0.000     -13.161      -6.957
n_levels_completed                -0.4843      0.175     -2.760      0.006      -0.828      -0.140
n_missions_completed               0.0231      0.007      3.174      0.002       0.009       0.037
n_package_info_offers_viewed      -0.0052      0.001     -4.340      0.000      -0.008      -0.003
sum_payments_package_key           2.6761      0.018    146.999      0.000       2.640       2.712
n_package_tips_offers_viewed      -0.2028      0.033     -6.139      0.000      -0.267      -0.138
n_sessions_ended                  -0.4926      0.149     -3.306      0.001      -0.785      -0.201
total_session_duration          6.289e-07   3.99e-08     15.771      0.000    5.51e-07    7.07e-07
n_sessions_started                 0.4869      0.148      3.296      0.001       0.197       0.776
min_session_start_player_level     2.6709      0.331      8.073      0.000       2.022       3.319
n_time_skip_buys                   1.9207      0.186     10.344      0.000       1.557       2.285
n_viplevels_completed             -9.2122      1.058     -8.707      0.000     -11.286      -7.139
gdp_per_capita                     0.0001   3.55e-05      3.984      0.000    7.17e-05       0.000
ram_max                            0.0007      0.000      2.301      0.021       0.000       0.001
avg_country_spend_30d              0.2734      0.052      5.262      0.000       0.172       0.375
avg_country_spend_30d_first3d     -0.7325      0.122     -6.009      0.000      -0.971      -0.494
nunique_iaps_bought                2.5418      0.403      6.308      0.000       1.752       3.332
time_to_first_purchase         -4.686e-05    1.4e-05     -3.355      0.001   -7.42e-05   -1.95e-05
time_to_last_purchase           9.197e-05   1.17e-05      7.848      0.000     6.9e-05       0.000
==============================================================================
Omnibus:                   185316.881   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.000   Jarque-Bera (JB):      16332356042.243
Skew:                          31.219   Prob(JB):                         0.00
Kurtosis:                    2370.942   Cond. No.                     1.92e+08
====

## Lineárna regresia za pomoci scikit learn

In [29]:
regr = linear_model.LinearRegression(fit_intercept=True)
regr.fit(X_train,Y_train)
y_pred = regr.predict(X_test)

In [30]:
#score parameters
print(f"R^2 skóre pre tréningový dataset: {regr.score(X_train,Y_train)}")
print(f"R^2 skóre pre testovací dataet: {regr.score(X_test,Y_test)}")
print(f"MAE: {mean_absolute_error(Y_test,y_pred)}")
print(f"MSE: {mean_squared_error(Y_test,y_pred)}")
print(f"RMSE: {math.sqrt(mean_squared_error(Y_test,y_pred))}")

R^2 skóre pre tréningový dataset: 0.4285749032148223
R^2 skóre pre testovací dataet: 0.31797368247379243
MAE: 38.046701899351845
MSE: 26896.781027734618
RMSE: 164.00238116483132


In [31]:
#lineárna regresia po vyhodení nevýznamných premenných
regr = linear_model.LinearRegression(fit_intercept=True)
regr.fit(adjusted_X_train,Y_train)
y_pred = regr.predict(adjusted_X_test)

In [32]:
print(f"R^2 skóre pre tréningový dataset: {regr.score(adjusted_X_train,Y_train)}")
print(f"R^2 skóre pre testovací dataet: {regr.score(adjusted_X_test,Y_test)}")
print(f"MAE: {mean_absolute_error(Y_test,y_pred)}")
print(f"MSE: {mean_squared_error(Y_test,y_pred)}")
print(f"RMSE: {math.sqrt(mean_squared_error(Y_test,y_pred))}")

R^2 skóre pre tréningový dataset: 0.4283600332826919
R^2 skóre pre testovací dataet: 0.32087190966733226
MAE: 37.177736420424665
MSE: 26782.48487787927
RMSE: 163.6535513757012


## Logaritmická regresia s využitím statsmodels

In [33]:
numeric_variables_buyers_log = []
for i in range(0,len(numeric_variables_buyers)):
    numeric_variables_buyers_log.append(math.log(numeric_variables_buyers.iloc[i,45]))

In [34]:
numeric_variables_buyers_log_test = []
for i in range(0,len(buyers_test_numeric_variables)):
    numeric_variables_buyers_log_test.append(math.log(buyers_test_numeric_variables.iloc[i,45]))

In [35]:
log_y_train = {"sum_payments_package_key_ltv": numeric_variables_buyers_log}
log_y_train = pd.DataFrame(log_y_train)

In [36]:
log_y_test = {"sum_payments_package_key_ltv": numeric_variables_buyers_log_test}
log_y_test = pd.DataFrame(log_y_test)

In [37]:
dep_var = log_y_train
exp_var = X_train
exp_var = sm.add_constant(exp_var) 

In [38]:
ols = sm.OLS(dep_var.astype(float),exp_var.astype(float))
ols_fit = ols.fit()
ols_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     sum_payments_package_key_ltv   R-squared:                       0.491
Model:                                      OLS   Adj. R-squared:                  0.490
Method:                           Least Squares   F-statistic:                     1495.
Date:                          Wed, 18 May 2022   Prob (F-statistic):               0.00
Time:                                  16:57:03   Log-Likelihood:            -1.0989e+05
No. Observations:                         69858   AIC:                         2.199e+05
Df Residuals:                             69812   BIC:                         2.203e+05
Df Model:                                    45                                         
Covariance Type:                      nonrobust                                         
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
const                                    -0.2044      0.482     -0.424      0.672      -1.150       0.741
n_ad_reward_claims                       -0.0050      0.001     -4.145      0.000      -0.007      -0.003
n_ad_reward_fails                        -0.0004      0.003     -0.130      0.897      -0.007       0.006
n_ads_watched                            -0.0008      0.000     -6.873      0.000      -0.001      -0.001
n_battlepass_lvls_finished                0.0230      0.003      7.852      0.000       0.017       0.029
n_instant_awards_claims                   0.0132      0.019      0.683      0.495      -0.025       0.051
n_extra_challenge_buys                   -0.3046      0.022    -14.159      0.000      -0.347      -0.262
connected_fb                             -0.0465      0.025     -1.827      0.068      -0.096       0.003
connect_fb_attempt                        0.0257      0.014      1.848      0.065      -0.002       0.053
n_friend_installs                         0.2496      0.118      2.117      0.034       0.019       0.481
n_accept_all_friends                     -0.0174      0.009     -1.893      0.058      -0.036       0.001
n_accept_friend                          -0.0554      0.014     -4.034      0.000      -0.082      -0.028
n_clicked_add_friends                    -0.0032      0.005     -0.652      0.515      -0.013       0.007
n_friend_searches                         0.0209      0.027      0.767      0.443      -0.033       0.074
n_levels_completed                       -0.0156      0.001    -10.635      0.000      -0.018      -0.013
n_calendar_login_days                     0.0019      0.001      3.215      0.001       0.001       0.003
n_missions_completed                      0.0002   8.32e-05      2.119      0.034    1.32e-05       0.000
n_package_info_offers_viewed             -0.0001   9.65e-06    -10.783      0.000      -0.000   -8.52e-05
n_payments_package_key                   -0.0403      0.006     -6.986      0.000      -0.052      -0.029
sum_payments_package_key                  0.0042      0.000     25.554      0.000       0.004       0.005
n_package_tips_offers_viewed              0.0028      0.000     10.209      0.000       0.002       0.003
rated_game                               -0.8169      0.413     -1.977      0.048      -1.627      -0.007
n_remove_ad_clicks                       -0.0112      0.012     -0.951      0.342      -0.034       0.012
n_sessions_ended                         -0.0057      0.001     -4.740      0.000      -0.008      -0.003
total_session_duration                 5.775e-09   3.48e-10     16.575      0.000    5.09e-09    6.46e-09
max_s

In [39]:
#logaritmická regresia po vyhodení nevýznamných premenných
dep_var = log_y_train
exp_var = X_train.drop(["n_ad_reward_fails","n_instant_awards_claims","connected_fb","connect_fb_attempt","n_accept_all_friends","n_clicked_add_friends","n_friend_searches","n_remove_ad_clicks","n_time_skip_buys","avg_country_spend_30d","nunique_countries","first_login_country_is_mfreq"],axis=1)
exp_var = sm.add_constant(exp_var) #bola aj tak nevyznamna a zvysilo to kvalitu modelu plus umoznilo robit predict cez tuto kniznicu

In [40]:
ols = sm.OLS(dep_var.astype(float),exp_var.astype(float))
ols_fit = ols.fit()
ols_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     sum_payments_package_key_ltv   R-squared:                       0.491
Model:                                      OLS   Adj. R-squared:                  0.490
Method:                           Least Squares   F-statistic:                     2038.
Date:                          Wed, 18 May 2022   Prob (F-statistic):               0.00
Time:                                  16:57:04   Log-Likelihood:            -1.0990e+05
No. Observations:                         69858   AIC:                         2.199e+05
Df Residuals:                             69824   BIC:                         2.202e+05
Df Model:                                    33                                         
Covariance Type:                      nonrobust                                         
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
const                                    -0.1700      0.469     -0.362      0.717      -1.089       0.749
n_ad_reward_claims                       -0.0052      0.001     -4.361      0.000      -0.007      -0.003
n_ads_watched                            -0.0008      0.000     -6.875      0.000      -0.001      -0.001
n_battlepass_lvls_finished                0.0233      0.003      7.971      0.000       0.018       0.029
n_extra_challenge_buys                   -0.3117      0.021    -14.725      0.000      -0.353      -0.270
n_friend_installs                         0.2476      0.118      2.100      0.036       0.016       0.479
n_accept_friend                          -0.0564      0.014     -4.110      0.000      -0.083      -0.030
n_levels_completed                       -0.0154      0.001    -10.566      0.000      -0.018      -0.013
n_calendar_login_days                     0.0019      0.001      3.113      0.002       0.001       0.003
n_missions_completed                      0.0002   8.29e-05      1.880      0.060   -6.68e-06       0.000
n_package_info_offers_viewed             -0.0001   9.61e-06    -10.876      0.000      -0.000   -8.57e-05
n_payments_package_key                   -0.0402      0.006     -6.980      0.000      -0.052      -0.029
sum_payments_package_key                  0.0042      0.000     25.502      0.000       0.004       0.005
n_package_tips_offers_viewed              0.0028      0.000     10.320      0.000       0.002       0.003
rated_game                               -0.8160      0.413     -1.975      0.048      -1.626      -0.006
n_sessions_ended                         -0.0057      0.001     -4.703      0.000      -0.008      -0.003
total_session_duration                 5.788e-09   3.48e-10     16.645      0.000    5.11e-09    6.47e-09
max_session_end_player_level              0.0152      0.003      5.916      0.000       0.010       0.020
n_sessions_started                        0.0053      0.001      4.368      0.000       0.003       0.008
min_session_start_viplevel                0.3674      0.142      2.590      0.010       0.089       0.646
min_session_start_player_level            0.0065      0.003      2.309      0.021       0.001       0.012
n_viplevels_completed                     0.3080      0.009     35.430      0.000       0.291       0.325
bigmac_dollar_price                       0.0334      0.007      4.802      0.000       0.020       0.047
gdp_per_capita                          1.04e-05   4.31e-07     24.150      0.000    9.56e-06    1.12e-05
ram_max                                3.545e-05   2.63e-06     13.459      0.000    3.03e-05    4.06e-05
scree

## Logaritmická regresia s využitím scikit learn

In [41]:
log_regr = linear_model.LinearRegression()
log_regr.fit(X_train,log_y_train)
log_y_pred = log_regr.predict(X_test)

In [42]:
print(f"R^2 skóre pre trénovací dataset: {log_regr.score(X_train,log_y_train)}")
print(f"R^2 skóre pre testovací dataset: {log_regr.score(X_test,log_y_test)}")
print(f"MAE: {mean_absolute_error(log_y_test,log_y_pred)}")
print(f"MSE: {mean_squared_error(log_y_test,log_y_pred)}")
print(f"RMSE: {math.sqrt(mean_squared_error(log_y_test,log_y_pred))}")

R^2 skóre pre trénovací dataset: 0.49074346359152077
R^2 skóre pre testovací dataset: 0.4329984884503193
MAE: 0.927477296373004
MSE: 1.5069509691276135
RMSE: 1.2275793127646024


In [43]:
#logaritmicka regresia po vyhodení nevýznamných premenných
adj_for_log_X_test = X_test.drop(["n_ad_reward_fails","n_instant_awards_claims","connected_fb","connect_fb_attempt","n_accept_all_friends","n_clicked_add_friends","n_friend_searches","n_remove_ad_clicks","n_time_skip_buys","avg_country_spend_30d","nunique_countries","first_login_country_is_mfreq"],axis=1)
adj_for_log_X_train = X_train.drop(["n_ad_reward_fails","n_instant_awards_claims","connected_fb","connect_fb_attempt","n_accept_all_friends","n_clicked_add_friends","n_friend_searches","n_remove_ad_clicks","n_time_skip_buys","avg_country_spend_30d","nunique_countries","first_login_country_is_mfreq"],axis=1)
log_regr = linear_model.LinearRegression()
log_regr.fit(adj_for_log_X_train,log_y_train)
log_y_pred = log_regr.predict(adj_for_log_X_test)

In [44]:
print(f"R^2 skóre pre trénovací dataset: {log_regr.score(adj_for_log_X_train,log_y_train)}")
print(f"R^2 skóre pre testovací dataset: {log_regr.score(adj_for_log_X_test,log_y_test)}")
print(f"MAE: {mean_absolute_error(log_y_test,log_y_pred)}")
print(f"MSE: {mean_squared_error(log_y_test,log_y_pred)}")
print(f"RMSE:{math.sqrt(mean_squared_error(log_y_test,log_y_pred))}")

R^2 skóre pre trénovací dataset: 0.4906413889100434
R^2 skóre pre testovací dataset: 0.4325305306110965
MAE: 0.9278944204517513
MSE: 1.5081946863046638
RMSE:1.228085781329897


In [45]:
numeric_variables_buyers.shape

(69858, 47)